# Mô phỏng Hệ thống Buffet - System Performance Evaluation

Notebook này mô phỏng hệ thống buffet với các quầy thức ăn khác nhau sử dụng các kỷ luật hàng đợi khác nhau (FCFS, SJF, ROS).

## Hướng dẫn sử dụng:
1. Chạy tất cả các cells từ trên xuống dưới
2. Ở cell cuối cùng, chọn config muốn chạy (all_ros, all_fcfs, all_sjf, best_combination_normal, best_combination_rush_hour)
3. Kết quả sẽ hiển thị các chỉ số hiệu suất của hệ thống

## Các chỉ số đánh giá:
- Thời gian chờ trung bình tại mỗi quầy
- Thời gian trung bình trong hệ thống
- Xác suất bị chặn (Balking)
- Xác suất rời hàng (Reneging)

In [1]:
%pip install simpy numpy matplotlib -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
import simpy
import random
import numpy as np
import heapq
from abc import ABC, abstractmethod

In [ ]:
################ Customer ####################
class Customer:
    """
    Đại diện cho một 'thực thể' (entity) di chuyển trong hệ thống.
    Chủ yếu là một cấu trúc dữ liệu để lưu trữ trạng thái và thuộc tính của từng khách hàng.
    """
    def __init__(self, id: int, arrival_gate: int, arrival_time: float,
                 customer_type: str, patience_time: float, service_times: dict):
        self.id = id
        self.arrival_gate = arrival_gate
        self.arrival_time = arrival_time
        self.customer_type = customer_type
        self.patience_time = patience_time
        self.service_times = service_times # {station_name: service_time}
        self.reneged = False # Mặc định là chưa bỏ đi
        self.start_wait_time = 0.0 # Thời gian bắt đầu chờ tại một quầy cụ thể
        self.served_event = None # SimPy Event để quản lý việc phục vụ
        
################ Analysis #######################
class Analysis:
    """
    Tách biệt logic thu thập và xử lý số liệu ra khỏi mô phỏng.
    """
    def __init__(self):
        # --- Bộ đếm (Counters) ---
        self.total_arrivals = 0      # [cite: 163]
        self.total_exits = 0         # [cite: 164]

        # Khách bỏ đi vì không gian K đầy (Balking) [cite: 166, 222]
        self.total_balked = 0

        # Khách bỏ đi vì chờ quá lâu (Reneging)
        self.total_reneged = 0

        # --- Dữ liệu thô (Raw Data) ---
        # {'Meat': [t1, t2], 'Seafood': [t3, ...]}
        self.wait_times = {}         # [cite: 167]
        self.system_times = []       # List thời gian khách ở trong hệ thống [cite: 168]

        # {'Meat': 5, 'Seafood': 2} (số lần bị chặn)
        self.blocking_events = {}    # [cite: 169]
        self.reneging_events = {}

        # --- Thống kê đã tính (Calculated Statistics) ---
        self.avg_wait_time_per_station = {}
        self.avg_system_time = 0.0
        self.blocking_probability_per_station = {}
        self.reneging_probability_per_station = {}
        self.total_attempts_per_station = {} # Cần để tính xác suất

    def add_station(self, station_name):
        """Đăng ký station để theo dõi số liệu."""
        if station_name not in self.wait_times:
            self.wait_times[station_name] = []
            self.blocking_events[station_name] = 0
            self.total_attempts_per_station[station_name] = 0
            self.reneging_events[station_name] = 0

    def record_arrival(self):
        """[cite: 171]"""
        self.total_arrivals += 1

    def record_exit(self, system_time):
        """[cite: 172]"""
        self.total_exits += 1
        self.system_times.append(system_time)

    def record_attempt(self, station_name):
        """Ghi nhận khi khách *cố gắng* vào một quầy."""
        self.total_attempts_per_station[station_name] += 1

    def record_wait_time(self, station_name, wait):
        """[cite: 173]"""
        self.wait_times[station_name].append(wait)

    def record_blocking_event(self, station_name):
        """Ghi nhận khi khách bị chặn (Balking)[cite: 174, 222]."""
        if station_name not in self.blocking_events:
            self.blocking_events[station_name] = 0
        if station_name not in self.total_attempts_per_station:
            self.total_attempts_per_station[station_name] = 0
        self.blocking_events[station_name] += 1

    def record_customer_balk(self):
        """Ghi nhận tổng số khách bỏ về do hết chỗ K."""
        self.total_balked += 1

    def record_reneging_event(self, station_name):
        """Ghi nhận khi khách rời hàng đợi (Reneging)."""
        if station_name not in self.reneging_events:
            self.reneging_events[station_name] = 0
        self.reneging_events[station_name] += 1
        self.total_reneged += 1

    def calculate_statistics(self):
        """
        Tính toán các chỉ số có ý nghĩa từ dữ liệu thô. [cite: 248, 249]
        """
        if self.system_times:
            self.avg_system_time = np.mean(self.system_times)
        else:
            self.avg_system_time = 0.0

        for station, times in self.wait_times.items():
            if times:
                self.avg_wait_time_per_station[station] = np.mean(times)
            else:
                self.avg_wait_time_per_station[station] = 0.0

        for station, blocked_count in self.blocking_events.items():
            attempts = self.total_attempts_per_station.get(station, 0)
            if attempts > 0:
                self.blocking_probability_per_station[station] = blocked_count / attempts
            else:
                self.blocking_probability_per_station[station] = 0.0

        for station, reneged_count in self.reneging_events.items():
            attempts = self.total_attempts_per_station.get(station, 0)
            if attempts > 0:
                self.reneging_probability_per_station[station] = reneged_count / attempts
            else:
                self.reneging_probability_per_station[station] = 0.0

    def print_report(self):
        """Định dạng và in kết quả đã tính. """
        print("--- BAO CAO MO PHONG ---")
        print(f"Tong so khach den: {self.total_arrivals}")
        print(f"Tong so khach thoat: {self.total_exits}")
        print(f"Tong so khach bo ve (Balked - het cho K): {self.total_balked}")
        print(f"Tong so khach bo ve (Reneged - mat kien nhan): {self.total_reneged}")

        # overall_balking_rate = (
        #     self.total_balked / self.total_arrivals
        #     if self.total_arrivals else 0.0
        # )
        # overall_reneging_rate = (
        #     self.total_reneged / self.total_arrivals
        #     if self.total_arrivals else 0.0
        # )
        # print(f"Tyle khach bi chan vi day K: {overall_balking_rate:.2%}")
        # print("Tyle khach reneging khi het DEFAULT_PATIENCE_TIME: "
        #       f"{overall_reneging_rate:.2%}")

        print(f"\nThoi gian trung binh trong he thong: {self.avg_system_time:.2f}")

        print("\nThoi gian cho trung binh tai quay:")
        # In tất cả stations, kể cả không có wait time
        all_stations = set(self.wait_times.keys()) | set(self.total_attempts_per_station.keys())
        station_order = ['Meat', 'Seafood', 'Dessert', 'Fruit']
        for station in station_order:
            time = self.avg_wait_time_per_station.get(station, 0.0)
            attempts = self.total_attempts_per_station.get(station, 0)
            wait_count = len(self.wait_times.get(station, []))
            print(f"  - {station:<10}: {time:.4f} (attempts: {attempts}, wait_records: {wait_count})")

        print("\nXac suat bi chan (Balking):")
        for station, prob in self.blocking_probability_per_station.items():
            print(f"  - {station:<10}: {prob:.4%}")

        print("\nXac suat bi chan (Reneging - Het patience):")
        for station, prob in self.reneging_probability_per_station.items():
            print(f"  - {station:<10}: {prob:.4%}")

# ========== Base Queue System ==========
class BaseQueueSystem(ABC):
    """
    Lớp cơ sở trừu tượng (Abstract Base Class) cho tất cả mô hình hàng đợi.
    """
    def __init__(self, env: simpy.Environment, num_servers: int,
                 avg_service_time: float, analyzer: Analysis, station_name: str):
        self.env = env
        self.num_servers = num_servers
        self.avg_service_time = avg_service_time
        self.analyzer = analyzer
        self.station_name = station_name

    @abstractmethod
    def serve(self, customer: Customer):
        """Tiến trình trừu tượng cho việc phục vụ khách hàng."""
        pass

# ========== FCFS Model ==========
class FCFSModel(BaseQueueSystem):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.servers = simpy.Resource(self.env, capacity=self.num_servers)

    def serve(self, customer: Customer):
        patience_remaining = customer.patience_time

        if patience_remaining <= 0:
            customer.reneged = True
            self.analyzer.record_reneging_event(self.station_name)
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)
            return

        with self.servers.request() as req:
            results = yield req | self.env.timeout(patience_remaining)
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)

            if req not in results:
                customer.reneged = True
                self.analyzer.record_reneging_event(self.station_name)
                return

            base_service_time = customer.service_times.get(
                self.station_name, self.avg_service_time
            )

            if customer.customer_type == 'indulgent':
                base_service_time *= 2.0

            actual_service_time = random.expovariate(1.0 / base_service_time)
            yield self.env.timeout(actual_service_time)

# ========== SJF Model ==========
STARVATION_THRESHOLD = 10.0

class SJFModel(BaseQueueSystem):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.servers = simpy.Container(self.env, capacity=self.num_servers, init=self.num_servers)
        self.wait_list = []
        self.customer_arrival = self.env.event()
        self.env.process(self.server_manager())

    def serve(self, customer: Customer):
        service_time = customer.service_times.get(self.station_name, self.avg_service_time)

        if customer.customer_type == 'indulgent':
            service_time *= 2.0

        if customer.customer_type == 'erratic':
            erratic_delay = 0.2
            for i, (prio, arr_time, waiting_customer) in enumerate(self.wait_list):
                if hasattr(waiting_customer, 'service_times'):
                    station_time = waiting_customer.service_times.get(
                        self.station_name, self.avg_service_time
                    )
                    waiting_customer.service_times[self.station_name] = station_time + erratic_delay

        heapq.heappush(self.wait_list, (service_time, self.env.now, customer))

        if not self.customer_arrival.triggered:
            self.customer_arrival.succeed()

        customer.served_event = self.env.event()
        patience_remaining = customer.patience_time

        if patience_remaining <= 0:
            customer.reneged = True
            self.analyzer.record_reneging_event(self.station_name)
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)
            return

        results = yield customer.served_event | self.env.timeout(patience_remaining)

        if customer.served_event not in results:
            self.analyzer.record_reneging_event(self.station_name)
            customer.reneged = True
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)

        customer.served_event = None

    def server_manager(self):
        while True:
            if not self.wait_list:
                yield self.customer_arrival
                self.customer_arrival = self.env.event()
                continue

            customer = self.find_customer_to_serve()
            if not customer:
                yield self.customer_arrival
                self.customer_arrival = self.env.event()
                continue

            yield self.servers.get(1)
            self.env.process(self.run_service(customer))

    def find_customer_to_serve(self):
        while self.wait_list:
            (service_time, arrival_time, customer) = heapq.heappop(self.wait_list)

            if hasattr(customer, 'reneged') and customer.reneged:
                continue

            wait_time = self.env.now - arrival_time

            if wait_time > STARVATION_THRESHOLD:
                return customer

            return customer

        return None

    def run_service(self, customer: Customer):
        if hasattr(customer, 'reneged') and customer.reneged:
            yield self.servers.put(1)
            return

        wait_time = self.env.now - customer.start_wait_time
        self.analyzer.record_wait_time(self.station_name, wait_time)

        if hasattr(customer, 'served_event') and customer.served_event:
            customer.served_event.succeed()

        base_service_time = customer.service_times.get(
            self.station_name, self.avg_service_time
        )

        if customer.customer_type == 'indulgent':
            base_service_time *= 2.0

        actual_service_time = random.expovariate(1.0 / base_service_time)
        yield self.env.timeout(actual_service_time)
        yield self.servers.put(1)

        if not self.customer_arrival.triggered:
            self.customer_arrival.succeed()

# ========== ROS Model ==========
class ROSModel(BaseQueueSystem):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.servers = simpy.Container(self.env, capacity=self.num_servers, init=self.num_servers)
        self.wait_list = []
        self.customer_arrival = self.env.event()
        self.env.process(self.server_manager())

    def serve(self, customer: Customer):
        self.wait_list.append(customer)

        if not self.customer_arrival.triggered:
            self.customer_arrival.succeed()

        customer.served_event = self.env.event()
        patience_remaining = customer.patience_time

        if patience_remaining <= 0:
            customer.reneged = True
            self.analyzer.record_reneging_event(self.station_name)
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)
            return

        results = yield customer.served_event | self.env.timeout(patience_remaining)

        if customer.served_event not in results:
            self.analyzer.record_reneging_event(self.station_name)
            customer.reneged = True
            wait_time = self.env.now - customer.start_wait_time
            self.analyzer.record_wait_time(self.station_name, wait_time)

        customer.served_event = None

    def server_manager(self):
        while True:
            if not self.wait_list:
                yield self.customer_arrival
                self.customer_arrival = self.env.event()
                continue

            customer = self.find_customer_to_serve()
            if not customer:
                yield self.customer_arrival
                self.customer_arrival = self.env.event()
                continue

            yield self.servers.get(1)
            self.env.process(self.run_service(customer))

    def find_customer_to_serve(self):
        while self.wait_list:
            idx = random.randrange(len(self.wait_list))
            customer = self.wait_list.pop(idx)

            if hasattr(customer, 'reneged') and customer.reneged:
                continue

            return customer

        return None

    def run_service(self, customer: Customer):
        if hasattr(customer, 'reneged') and customer.reneged:
            yield self.servers.put(1)
            return

        wait_time = self.env.now - customer.start_wait_time
        self.analyzer.record_wait_time(self.station_name, wait_time)

        if hasattr(customer, 'served_event') and customer.served_event:
            customer.served_event.succeed()

        base_service_time = customer.service_times.get(
            self.station_name, self.avg_service_time
        )

        if customer.customer_type == 'indulgent':
            base_service_time *= 2.0

        if customer.customer_type == 'erratic':
            erratic_delay = 0.2
            for waiting_customer in self.wait_list:
                if hasattr(waiting_customer, 'service_times'):
                    station_time = waiting_customer.service_times.get(
                        self.station_name, self.avg_service_time
                    )
                    waiting_customer.service_times[self.station_name] = station_time + erratic_delay

        actual_service_time = random.expovariate(1.0 / base_service_time)
        yield self.env.timeout(actual_service_time)
        yield self.servers.put(1)

        if not self.customer_arrival.triggered:
            self.customer_arrival.succeed()

# ========== Queue System Factory ==========
class QueueSystemFactory:
    def create_queue_model(self, env: simpy.Environment, config: dict,
                             analyzer: Analysis, station_name: str):
        discipline = config['discipline']
        num_servers = config['servers']
        avg_service_time = config['avg_service_time']

        common_args = (env, num_servers, avg_service_time, analyzer, station_name)

        if discipline == 'FCFS':
            return FCFSModel(*common_args)
        elif discipline == 'SJF':
            return SJFModel(*common_args)
        elif discipline == 'ROS':
            return ROSModel(*common_args)
        else:
            raise ValueError(f"Kỷ luật hàng đợi không xác định: {discipline}")

# ========== Food Station ==========
class FoodStation:
    def __init__(self, env: simpy.Environment, name: str,
                 capacity_K: int, analyzer: Analysis,
                 discipline_model: BaseQueueSystem, config=None):
        self.env = env
        self.name = name
        self.analyzer = analyzer
        self.discipline_model = discipline_model
        self.config = config
        self.capacity_K = capacity_K
        self.queue_space = simpy.Container(env, capacity=capacity_K, init=capacity_K)

    def serve(self, customer: Customer):
        self.analyzer.record_attempt(self.name)

        if self.queue_space.level == 0:
            customer.reneged = True
            self.analyzer.record_blocking_event(self.name)
            self.analyzer.record_customer_balk()
            return

        yield self.queue_space.get(1)

        if self.config:
            patience_factor = self.config.PATIENCE_TIME_FACTORS.get(
                customer.customer_type, 1.0
            )
            customer.patience_time = self.config.DEFAULT_PATIENCE_TIME * patience_factor

        customer.start_wait_time = self.env.now
        yield self.env.process(self.discipline_model.serve(customer))
        yield self.queue_space.put(1)

# ========== Buffet System ==========
class BuffetSystem:
    def __init__(self, env: simpy.Environment, analyzer: Analysis, config):
        self.env = env
        self.analyzer = analyzer
        self.config = config

        seed = getattr(config, 'RANDOM_SEED', 42)
        random.seed(seed)

        self.stations = {}
        self.arrival_rates = config.ARRIVAL_RATES
        self.prob_matrices = config.PROB_MATRICES
        self.factory = QueueSystemFactory()

        for name, cfg in config.STATIONS.items():
            model = self.factory.create_queue_model(
                env=env, config=cfg, analyzer=analyzer, station_name=name
            )

            self.stations[name] = FoodStation(
                env=env, name=name, capacity_K=cfg['capacity_K'],
                analyzer=analyzer, discipline_model=model, config=config
            )
            self.analyzer.add_station(name)

    def generate_customers(self, gate_id):
        arrival_rate = self.arrival_rates[gate_id]

        while True:
            inter_arrival_time = random.expovariate(arrival_rate)
            yield self.env.timeout(inter_arrival_time)

            customer_id = self.analyzer.total_arrivals
            self.analyzer.record_arrival()

            customer_service_times = {}
            for station, base_time in self.config.DEFAULT_SERVICE_TIMES.items():
                customer_service_times[station] = random.uniform(base_time * 0.5, base_time * 1.5)

            customer_types = list(self.config.CUSTOMER_TYPE_DISTRIBUTION.keys())
            customer_weights = list(self.config.CUSTOMER_TYPE_DISTRIBUTION.values())
            customer_type = random.choices(customer_types, weights=customer_weights, k=1)[0]

            patience_factor = self.config.PATIENCE_TIME_FACTORS.get(customer_type, 1.0)
            patience_time = self.config.DEFAULT_PATIENCE_TIME * patience_factor

            new_customer = Customer(
                id=customer_id, arrival_gate=gate_id, arrival_time=self.env.now,
                customer_type=customer_type, patience_time=patience_time,
                service_times=customer_service_times
            )

            self.env.process(self.customer_lifecycle(new_customer))

    def customer_lifecycle(self, customer: Customer):
        visited_stations = set() if customer.customer_type == 'indulgent' else None

        station_name, no_available = self.choose_initial_section(customer.arrival_gate)
        if station_name is None:
            if no_available:
                customer.reneged = True
            return

        while station_name is not None:
            station = self.stations[station_name]

            if visited_stations is not None:
                visited_stations.add(station_name)

            yield self.env.process(station.serve(customer))

            if customer.reneged:
                break

            next_station, reason = self.choose_next_action(customer, visited_stations)
            if next_station is None:
                if reason == 'no_available':
                    customer.reneged = True
                break
            station_name = next_station

        if not customer.reneged:
            system_time = self.env.now - customer.arrival_time
            self.analyzer.record_exit(system_time)

    def choose_initial_section(self, gate_id):
        prob_map = self.prob_matrices['initial'][gate_id]
        return self._select_station_with_capacity(prob_map)

    def choose_next_action(self, customer: Customer, visited_stations):
        prob_map = self.prob_matrices['next_action']
        action = random.choices(
            list(prob_map.keys()), weights=list(prob_map.values()), k=1
        )[0]

        if action == 'Exit':
            return None, 'exit'

        prob_map_transition = self.prob_matrices['transition']
        next_station, no_available = self._select_station_with_capacity(
            prob_map_transition, visited_stations
        )
        if next_station is None and no_available:
            return None, 'no_available'
        return next_station, None

    def _select_station_with_capacity(self, prob_map, visited_stations=None):
        current_probs = {}
        for station, prob in prob_map.items():
            if visited_stations is not None and station in visited_stations:
                continue
            current_probs[station] = prob

        if not current_probs:
            return None, False

        full_attempts = []
        full_set = set()
        while True:
            active_stations = [s for s, p in current_probs.items() if p > 0]
            if not active_stations:
                if full_attempts:
                    self._record_balking_for_stations(full_attempts)
                    return None, True
                return None, False

            weights = [current_probs[s] for s in active_stations]
            chosen = random.choices(active_stations, weights=weights, k=1)[0]

            if self.stations[chosen].queue_space.level > 0:
                return chosen, False

            full_attempts.append(chosen)
            full_set.add(chosen)
            prob_loss = current_probs[chosen]
            current_probs[chosen] = 0.0

            remaining = [s for s in current_probs if s not in full_set]
            if not remaining:
                self._record_balking_for_stations(full_attempts)
                return None, True

            share = prob_loss / len(remaining)
            for station in remaining:
                current_probs[station] += share

    def _record_balking_for_stations(self, stations):
        unique = set(stations)
        for station_name in unique:
            self.analyzer.record_attempt(station_name)
            self.analyzer.record_blocking_event(station_name)
        if unique:
            self.analyzer.record_customer_balk()

    def run(self, until_time):
        for gate_id in self.arrival_rates.keys():
            self.env.process(self.generate_customers(gate_id))

        print(f"--- Bat dau mo phong (Until={until_time}) ---")
        self.env.run(until=until_time)
        print("--- Ket thuc mo phong ---")

NameError: name 'Customer' is not defined

In [ ]:
# ========== CONFIG DEFINITIONS ==========
# Tạo một class Config để chứa các config
class Config:
    def __init__(self, name, seed, arrival_rates, stations, prob_matrices, 
                 until_time=1000.0, default_patience_time=10.0,
                 customer_type_distribution=None, patience_time_factors=None,
                 erratic_delay_amount=0.2, default_service_times=None):
        self.name = name
        self.RANDOM_SEED = seed
        self.UNTIL_TIME = until_time
        self.ARRIVAL_RATES = arrival_rates
        self.STATIONS = stations
        self.PROB_MATRICES = prob_matrices
        self.DEFAULT_PATIENCE_TIME = default_patience_time
        self.CUSTOMER_TYPE_DISTRIBUTION = customer_type_distribution or {
            'normal': 0.70, 'indulgent': 0.10, 'impatient': 0.15, 'erratic': 0.05
        }
        self.PATIENCE_TIME_FACTORS = patience_time_factors or {
            'normal': 1.0, 'indulgent': 1.0, 'impatient': 0.5, 'erratic': 1.0
        }
        self.ERRATIC_DELAY_AMOUNT = erratic_delay_amount
        self.DEFAULT_SERVICE_TIMES = default_service_times or {
            'Meat': 0.7, 'Seafood': 0.5, 'Dessert': 0.8, 'Fruit': 0.3
        }

# Định nghĩa các config
CONFIGS = {}

# Config: all_ros
CONFIGS['all_ros'] = Config(
    name='all_ros',
    seed=100,
    arrival_rates={0: 12, 1: 10},
    stations={
        'Meat': {'servers': 5, 'capacity_K': 10, 'discipline': 'ROS', 'avg_service_time': 0.5},
        'Seafood': {'servers': 7, 'capacity_K': 10, 'discipline': 'ROS', 'avg_service_time': 0.3},
        'Dessert': {'servers': 7, 'capacity_K': 10, 'discipline': 'ROS', 'avg_service_time': 0.5},
        'Fruit': {'servers': 10, 'capacity_K': 10, 'discipline': 'ROS', 'avg_service_time': 0.3}
    },
    prob_matrices={
        'initial': {
            0: {'Meat': 0.4, 'Seafood': 0.3, 'Dessert': 0.2, 'Fruit': 0.2},
            1: {'Meat': 0.3, 'Seafood': 0.4, 'Dessert': 0.15, 'Fruit': 0.15}
        },
        'next_action': {'More': 0.7, 'Exit': 0.3},
        'transition': {'Meat': 0.25, 'Seafood': 0.25, 'Dessert': 0.25, 'Fruit': 0.25}
    }
)

# Config: all_fcfs
CONFIGS['all_fcfs'] = Config(
    name='all_fcfs',
    seed=200,
    arrival_rates={0: 6, 1: 8},
    stations={
        'Meat': {'servers': 5, 'capacity_K': 10, 'discipline': 'FCFS', 'avg_service_time': 0.5},
        'Seafood': {'servers': 7, 'capacity_K': 10, 'discipline': 'FCFS', 'avg_service_time': 0.3},
        'Dessert': {'servers': 7, 'capacity_K': 10, 'discipline': 'FCFS', 'avg_service_time': 0.5},
        'Fruit': {'servers': 10, 'capacity_K': 10, 'discipline': 'FCFS', 'avg_service_time': 0.3}
    },
    prob_matrices={
        'initial': {
            0: {'Meat': 0.4, 'Seafood': 0.3, 'Dessert': 0.2, 'Fruit': 0.2},
            1: {'Meat': 0.3, 'Seafood': 0.4, 'Dessert': 0.15, 'Fruit': 0.15}
        },
        'next_action': {'More': 0.7, 'Exit': 0.3},
        'transition': {'Meat': 0.25, 'Seafood': 0.25, 'Dessert': 0.25, 'Fruit': 0.25}
    }
)

# Config: all_sjf
CONFIGS['all_sjf'] = Config(
    name='all_sjf',
    seed=300,
    arrival_rates={0: 12, 1: 10},
    stations={
        'Meat': {'servers': 5, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.5},
        'Seafood': {'servers': 7, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.3},
        'Dessert': {'servers': 7, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.5},
        'Fruit': {'servers': 10, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.3}
    },
    prob_matrices={
        'initial': {
            0: {'Meat': 0.4, 'Seafood': 0.3, 'Dessert': 0.2, 'Fruit': 0.2},
            1: {'Meat': 0.3, 'Seafood': 0.4, 'Dessert': 0.15, 'Fruit': 0.15}
        },
        'next_action': {'More': 0.7, 'Exit': 0.3},
        'transition': {'Meat': 0.25, 'Seafood': 0.25, 'Dessert': 0.25, 'Fruit': 0.25}
    }
)

# Config: best_combination_normal
CONFIGS['best_combination_normal'] = Config(
    name='best_combination_normal',
    seed=400,
    arrival_rates={0: 12, 1: 10},
    stations={
        'Meat': {'servers': 5, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.5},
        'Seafood': {'servers': 7, 'capacity_K': 10, 'discipline': 'SJF', 'avg_service_time': 0.3},
        'Dessert': {'servers': 7, 'capacity_K': 10, 'discipline': 'ROS', 'avg_service_time': 0.5},
        'Fruit': {'servers': 10, 'capacity_K': 10, 'discipline': 'FCFS', 'avg_service_time': 0.3}
    },
    prob_matrices={
        'initial': {
            0: {'Meat': 0.4, 'Seafood': 0.3, 'Dessert': 0.2, 'Fruit': 0.2},
            1: {'Meat': 0.3, 'Seafood': 0.4, 'Dessert': 0.15, 'Fruit': 0.15}
        },
        'next_action': {'More': 0.7, 'Exit': 0.3},
        'transition': {'Meat': 0.25, 'Seafood': 0.25, 'Dessert': 0.25, 'Fruit': 0.25}
    }
)

# Config: best_combination_rush_hour
CONFIGS['best_combination_rush_hour'] = Config(
    name='best_combination_rush_hour',
    seed=500,
    arrival_rates={0: 18, 1: 15},
    stations={
        'Meat': {'servers': 5, 'capacity_K': 15, 'discipline': 'SJF', 'avg_service_time': 0.5},
        'Seafood': {'servers': 7, 'capacity_K': 15, 'discipline': 'SJF', 'avg_service_time': 0.3},
        'Dessert': {'servers': 7, 'capacity_K': 15, 'discipline': 'ROS', 'avg_service_time': 0.5},
        'Fruit': {'servers': 10, 'capacity_K': 15, 'discipline': 'FCFS', 'avg_service_time': 0.3}
    },
    prob_matrices={
        'initial': {
            0: {'Meat': 0.4, 'Seafood': 0.3, 'Dessert': 0.2, 'Fruit': 0.2},
            1: {'Meat': 0.3, 'Seafood': 0.4, 'Dessert': 0.15, 'Fruit': 0.15}
        },
        'next_action': {'More': 0.7, 'Exit': 0.3},
        'transition': {'Meat': 0.25, 'Seafood': 0.25, 'Dessert': 0.25, 'Fruit': 0.25}
    }
)

print("Đã load các config:")
for name in CONFIGS.keys():
    print(f"  - {name}")


In [ ]:
# ========== FUNCTION TO RUN SIMULATION ==========
def run_simulation(config_name):
    """
    Chạy mô phỏng với config được chọn
    
    Args:
        config_name: Tên config ('all_ros', 'all_fcfs', 'all_sjf', 'best_combination_normal', 'best_combination_rush_hour')
    """
    if config_name not in CONFIGS:
        print(f"Lỗi: Config '{config_name}' không tồn tại!")
        print(f"Các config có sẵn: {list(CONFIGS.keys())}")
        return None
    
    config = CONFIGS[config_name]
    print(f"\n{'='*60}")
    print(f"Chạy mô phỏng với config: {config_name}")
    print(f"Seed: {config.RANDOM_SEED}")
    print(f"{'='*60}\n")
    
    # Khởi tạo môi trường
    env = simpy.Environment()
    
    # Khởi tạo bộ phân tích
    analyzer = Analysis()
    
    # Khởi tạo hệ thống buffet
    buffet = BuffetSystem(env, analyzer, config)
    
    # Chạy mô phỏng
    buffet.run(until_time=config.UNTIL_TIME)
    
    # Tính toán và in kết quả
    analyzer.calculate_statistics()
    analyzer.print_report()
    
    return analyzer


In [ ]:
# ========== CHỌN CONFIG VÀ CHẠY MÔ PHỎNG ==========
# Thay đổi tên config ở đây để chạy các scenario khác nhau:
# - 'all_ros': Tất cả quầy dùng ROS
# - 'all_fcfs': Tất cả quầy dùng FCFS
# - 'all_sjf': Tất cả quầy dùng SJF
# - 'best_combination_normal': Kết hợp tối ưu cho trường hợp bình thường
# - 'best_combination_rush_hour': Kết hợp tối ưu cho giờ cao điểm

config_name = 'all_ros'  # Thay đổi config ở đây

analyzer = run_simulation(config_name)